Sentiment Analysis

라이브러리

In [1]:
from os import name, sep
from nltk import tokenize
from nltk.corpus.reader import twitter
import pandas as pd
import numpy as np


from konlpy.tag import Okt
from konlpy.tag import Twitter
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from hanspell import spell_checker

from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

파일 불러오기, 훈련 데이터와 테스트 데이터 나누기(7:3)

In [2]:
df = pd.read_csv("C:/Users/User/OneDrive - 공주대학교/바탕 화면/review/맞춤법 정리(리뷰, 레이블).csv", encoding='cp949')

train_x, test_x, train_y, test_y = train_test_split(df["리뷰"],df["긍/부정"], test_size=0.3, random_state=10)

print(len(train_x), len(train_y))

print(len(test_x), len(test_y))

7461 7461
3198 3198


형태소 분리 및 TF-IDF

In [3]:
okt = Okt()
print(train_x[4])
print(okt.morphs(train_x[4]))

tfidf = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, tokenizer=okt.morphs, token_pattern=None)
tfidf.fit(train_x)
train_x_okt = tfidf.transform(train_x)
test_x_okt = tfidf.transform(test_x.values.astype('U'))

순두부찌개는 너무 짠맛만 나고.. 물 넣어서 전자레인지 돌려먹었어요 그냥 딱 쉰 김치 국물에 순두부에요 제육덮밥도.. 제가 리뷰만 너무 믿었나요 윤기 1도 없고 그냥 퍽퍽 그 자체에요 반찬도 참다 참다 쉰내가 너무 나서 말씀드렸더니 쉰 김치냄새 때문에 그렇다 먹어도 이상 없다 하시는데 제가 거기서 더 이상 뭔 얘길 하나요~ 진심 두 입씩 먹고 다 안 먹었습니다 .. 양은 많네요
['순두부찌개', '는', '너무', '짠맛', '만', '나고', '..', '물', '넣어서', '전자레인지', '돌려', '먹었어요', '그냥', '딱', '쉰', '김치', '국물', '에', '순두부', '에요', '제육', '덮', '밥', '도', '..', '제', '가', '리뷰', '만', '너무', '믿었나요', '윤기', '1', '도', '없고', '그냥', '퍽퍽', '그', '자체', '에요', '반찬', '도', '참다', '참다', '쉰내', '가', '너무', '나서', '말씀드렸더니', '쉰', '김치', '냄새', '때문', '에', '그렇다', '먹어도', '이상', '없다', '하시는데', '제', '가', '거기', '서', '더', '이상', '뭔', '얘길', '하나요', '~', '진심', '두', '입', '씩', '먹고', '다', '안', '먹었습니다', '..', '양은', '많네요']


로지스틱 회귀

In [4]:
sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}

rsv_okt = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist, n_iter=50, random_state=1, verbose=1)
rsv_okt.fit(train_x_okt, train_y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016A36CACE50>},
                   random_state=1, verbose=1)

점수 출력

In [5]:
print('최상 점수')
print(rsv_okt.best_score_)

print('최상 매개변수 값')
print(rsv_okt.best_params_)

print('test 점수')
print(rsv_okt.score(test_x_okt,test_y))

최상 점수
0.90068370896175
최상 매개변수 값
{'alpha': 0.0001001581395585897}
test 점수
0.9024390243902439


파일 저장